### NLCD RASTERS (SUMMER, 2025)

In [14]:
# Modules.
import os
from pathlib import Path
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import geopandas as gpd
from tqdm import tqdm
from rasterstats import zonal_stats

In [15]:
# Paths.
nlcd_tree_path = Path("data/raster/nlcd_raster/nlcd_tree_canopy_2023.tiff")
nlcd_impervious_path = Path("data/raster/nlcd_raster/nyc_ncld_impervious_2024.tiff")
nlcd_landcov_path = Path("data/raster/nlcd_raster/nyc_ncld_land_cover_2024.tiff")

tracts_path = Path("data/nyc_tracts_2020/nyc_tracts_2020.shp")

output_dir = Path("data/raster/processed")
output_dir.mkdir(parents = True, exist_ok = True)

tracts = gpd.read_file(tracts_path).to_crs(4326)

# Dissolved NYC boundary to Shapely.
nyc_boundary = tracts.union_all()

In [16]:
# Zonal statistics for NCLD.
def zonal_mean(rpath, gdf_or_geom):
    """Apply CRS zonal mean for tracts or city boundary."""
    with rasterio.open(rpath) as src:
        r_crs = src.crs

    if isinstance(gdf_or_geom, (gpd.GeoSeries, gpd.GeoDataFrame)):
        gdf = gdf_or_geom.to_crs(r_crs)
    else:
        gdf = gpd.GeoSeries([gdf_or_geom], crs = 4326).to_crs(r_crs)

    return zonal_stats(gdf, rpath, stats = ["mean"], nodata = np.nan)

In [17]:
# Print checks for the calculations.
print("Tree canopy zonal stats:")
tree = zonal_mean(nlcd_tree_path, tracts)
print("Impervious zonal stats:")
impervious = zonal_mean(nlcd_impervious_path, tracts)
print("Land cover zonal stats:")
landcover = zonal_mean(nlcd_landcov_path, tracts)

tracts["pct_tree_canopy"] = [t["mean"] for t in tree]
tracts["pct_impervious"] = [i["mean"] for i in impervious]
tracts["mean_landcover"] = [l["mean"] for l in landcover]

Tree canopy zonal stats:
Impervious zonal stats:
Land cover zonal stats:


In [19]:
tracts.columns = tracts.columns.str.upper()
tracts = tracts.rename(columns = {"GEOMETRY": "geometry"})

tracts.head()

,CTLABEL,BOROCODE,BORONAME,CT2020,BOROCT2020,CDELIGIBIL,NTANAME,NTA2020,CDTA2020,CDTANAME,GEOID,SHAPE_LENG,SHAPE_AREA,geometry,PCT_TREE_CANOPY,PCT_IMPERVIOUS,MEAN_LANDCOVER
0,1,1,Manhattan,000100,1000100,I,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,10833.043929,1.843005e+06,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",10.126984,1.825397,23.772487
1,14.01,1,Manhattan,001401,1001401,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((-73.98837 40.71645, -73.98754 40.716...",13.314286,1.476190,23.209524
2,14.02,1,Manhattan,001402,1001402,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,"POLYGON ((-73.98507 40.71908, -73.98423 40.718...",0.592000,1.176000,23.776000
3,18,1,Manhattan,001800,1001800,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001800,6391.921174,2.399277e+06,"POLYGON ((-73.98985 40.72052, -73.98972 40.720...",3.076305,1.253012,23.779116
4,22.01,1,Manhattan,002201,1002201,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061002201,5779.062607,1.740174e+06,"POLYGON ((-73.97875 40.71993, -73.97879 40.719...",6.089888,1.376404,23.589888


In [20]:
tracts.columns

Index(['CTLABEL', 'BOROCODE', 'BORONAME', 'CT2020', 'BOROCT2020', 'CDELIGIBIL',
       'NTANAME', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'PCT_TREE_CANOPY', 'PCT_IMPERVIOUS',
       'MEAN_LANDCOVER'],
      dtype='object')

In [21]:
tracts = tracts.drop(columns = ['CTLABEL', 'BOROCODE', 'BORONAME', 'CT2020',
                       'BOROCT2020', 'CDELIGIBIL', 'NTANAME', 'NTA2020',
                       'CDTA2020', 'CDTANAME', 'SHAPE_LENG', 'SHAPE_AREA'])

In [22]:
tracts.columns

Index(['GEOID', 'geometry', 'PCT_TREE_CANOPY', 'PCT_IMPERVIOUS',
       'MEAN_LANDCOVER'],
      dtype='object')

In [24]:
# Save as geojson.
geojson_out = output_dir.parent / "nlcd_calc_tracts.geojson"
geojson_out.parent.mkdir(parents = True, exist_ok = True)
tracts.to_file(geojson_out)
print("Saved:", geojson_out)

# Save as csv.
csv_out = output_dir.parent / "nlcd_calc_tracts.csv"
tracts.drop(columns = ["geometry"]).to_csv(csv_out, index = False)
print("Saved CSV:", csv_out)

Saved: data\raster\nlcd_calc_tracts.geojson
Saved CSV: data\raster\nlcd_calc_tracts.csv
